# Private Selection

Algorithm 1 from
**Liu, J., & Talwar, K. (2019, June). [Private selection from private candidates](https://arxiv.org/abs/1811.07971). In Proceedings of the 51st Annual ACM SIGACT Symposium on Theory of Computing (pp. 298-309).**

In [ ]:
import os
import secrets
import sys
from pathlib import Path

from IPython.display import Markdown, display

In [ ]:
sys.path.insert(0, "..")

from synthflow.__main__ import main

In [ ]:
# cryptographically secure pseudorandom number generator
# for private selection
CSPRNG = secrets.SystemRandom()

In [ ]:
# TODO: parameterized notebook

REAL_DATA_PATH = "../tests/data/birth_2019_nber_us_SINGLETON_SAMPLE160K.csv.gz"
RANDOM_SEED = 42
WANDB_GROUP = "test"
SUBSAMPLING = 0.4

In [ ]:
GEN_CONFIGS = list(Path("../../gen-config/").glob("*.json"))

GEN_CONFIGS

In [ ]:
os.environ["WANDB_MODE"] = "offline"
os.environ["WANB_SILENT"] = "true"
os.environ["PRIVBAYES_BIN"] = "../privbayes"
!wandb offline

In [ ]:
def private_selection():
    gen_config_path = str(CSPRNG.choice(GEN_CONFIGS))

    display(Markdown(f"# `{gen_config_path}`"))

    dp_acceptance_df = main(
        {
            "execute": True,
            "span": False,
            "parallel": False,
            "report": False,
            "evaluate": False,
            "<real-data-path>": REAL_DATA_PATH,
            "<gen-config-path>": gen_config_path,
            "--minimal": False,
            "--random-seed": RANDOM_SEED,
            "--wandb": True,
            "--wandb-group": WANDB_GROUP,
            "--subsampling": SUBSAMPLING,
            "--non-private-evaluation": False,
            "--verbose": True,
        }
    )

    acceptance_all_mask = dp_acceptance_df["name"] == "all"
    assert acceptance_all_mask.sum() == 1

    return dp_acceptance_df.loc[acceptance_all_mask, "check"].item(), gen_config_path

In [ ]:
history = []

check, gen_config_path = private_selection()
history.append(gen_config_path)

while not check:
    check, gen_config_path = private_selection()
    history.append(gen_config_path)

display(
    Markdown(f"# DONE after {len(history)} attempts, winner is `{gen_config_path}`")
)